In [1]:
pip install opendatasets

In [2]:
!git clone https://github.com/AndrewDiv/FCALC

Cloning into 'FCALC'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 89 (delta 41), reused 72 (delta 24), pack-reused 0
Receiving objects: 100% (89/89), 27.58 KiB | 3.06 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import opendatasets as od
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score

In [5]:
od.download("https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset")
od.download("https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset")
od.download("https://www.kaggle.com/datasets/adityakadiwal/water-potability")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: stevegeorgeparakal
Your Kaggle Key: ··········


100%|██████████| 4.11k/4.11k [00:00<00:00, 1.93MB/s]


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

 stevegeorgeparakal
Your Kaggle Key: ··········


100%|██████████| 67.4k/67.4k [00:00<00:00, 469kB/s]


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

 stevegeorgeparakal
Your Kaggle Key: ··········


100%|██████████| 251k/251k [00:00<00:00, 587kB/s]

In [6]:
file1 = ('/content/heart-attack-analysis-prediction-dataset/heart.csv')
heart_df = pd.read_csv(file1)

file2 = ('/content/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
stroke_df = pd.read_csv(file2)

file3 = ('/content/water-potability/water_potability.csv')
water_df = pd.read_csv(file3)

In [7]:
from FCALC import fcalc

# **Preprocessing**

In [8]:
heart_df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [9]:
heart_df.isnull().sum()

age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64

In [10]:
stroke_df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [11]:
stroke_df.drop('id', axis=1, inplace=True)

In [12]:
stroke_df.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [13]:
#Dealing with Categorical variables in stroke dataset
stroke_df['bmi'] = stroke_df['bmi'].fillna(stroke_df['bmi'].mean())

stroke_df['ever_married'] = stroke_df['ever_married'].replace(['No', 'Yes'], [0, 1])
stroke_df['Residence_type'] = stroke_df['Residence_type'].replace(['Rural', 'Urban'], [0, 1])

stroke_df = pd.get_dummies(stroke_df, columns=['gender', 'work_type', 'smoking_status'])

In [14]:
water_df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


In [15]:
water_df.isnull().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64

In [16]:
#Dealing with Categorical variables in water dataset
water_df['ph'] = water_df['ph'].fillna(water_df['ph'].mean())
water_df['Sulfate'] = water_df['Sulfate'].fillna(water_df['Sulfate'].mean())
water_df['Trihalomethanes'] = water_df['Trihalomethanes'].fillna(water_df['Trihalomethanes'].mean())

# **Decision Tree**

In [17]:
def tune_decision_tree(dataset, target_column_name):
    # Split the data into features (x) and the target variable (y)
    x = dataset.drop(columns=[target_column_name])
    y = dataset[target_column_name]

    # Split the data into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    # Create a DecisionTreeClassifier
    classifier = DecisionTreeClassifier()

    # Define a parameter grid for tuning
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    # Create a GridSearchCV object with 5-fold cross-validation
    grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy')

    # Fit the GridSearchCV object to the training data
    grid_search.fit(x_train, y_train)

    # Get the best parameters and estimator from the grid search
    best_params = grid_search.best_params_
    best_estimator = grid_search.best_estimator_

    # Assess the model using 5-fold cross-validation and print accuracy and F1 scores
    accuracy_scores = cross_val_score(best_estimator, x_train, y_train, cv=5, scoring='accuracy')
    f1_scores = cross_val_score(best_estimator, x_train, y_train, cv=5, scoring='f1_weighted')

    print("Best Parameters:", best_params)
    print("Cross-Validation Accuracy:", round(np.mean(accuracy_scores),3))
    print("Cross-Validation F1 Score:", round(np.mean(f1_scores),3))

    # Fit the best model on the entire training set
    best_estimator.fit(x_train, y_train)

    # Evaluate the model on the test set
    y_pred = best_estimator.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    test_f1_score = f1_score(y_test, y_pred, average='weighted')

    print("Test Set Accuracy:", round(test_accuracy,3))
    print("Test Set F1 Score:", round(test_f1_score,3))

In [18]:
tune_decision_tree(heart_df, 'output')

Best Parameters: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2}
Cross-Validation Accuracy: 0.756
Cross-Validation F1 Score: 0.758
Test Set Accuracy: 0.852
Test Set F1 Score: 0.852


In [19]:
tune_decision_tree(stroke_df, 'stroke')

Best Parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}
Cross-Validation Accuracy: 0.942
Cross-Validation F1 Score: 0.929
Test Set Accuracy: 0.931
Test Set F1 Score: 0.912


In [20]:
tune_decision_tree(water_df, 'Potability')

Best Parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2}
Cross-Validation Accuracy: 0.626
Cross-Validation F1 Score: 0.595
Test Set Accuracy: 0.61
Test Set F1 Score: 0.594


# **Random Forest**

In [21]:
def tune_random_forest(dataset, target_column_name):
    # Split the data into features (x) and the target variable (y)
    x = dataset.drop(columns=[target_column_name])
    y = dataset[target_column_name]

    # Split the data into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    # Create a RandomForestClassifier
    classifier = RandomForestClassifier()

    # Define a parameter grid for tuning
    param_grid = {
        'n_estimators': [25, 50, 100],
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    # Create a GridSearchCV object with 5-fold cross-validation
    grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy')

    # Fit the GridSearchCV object to the training data
    grid_search.fit(x_train, y_train)

    # Get the best parameters and estimator from the grid search
    best_params = grid_search.best_params_
    best_estimator = grid_search.best_estimator_

    # Assess the model using 5-fold cross-validation and print accuracy and F1 scores
    accuracy_scores = cross_val_score(best_estimator, x_train, y_train, cv=5, scoring='accuracy')
    f1_scores = cross_val_score(best_estimator, x_train, y_train, cv=5, scoring='f1_weighted')

    print("Best Parameters:", best_params)
    print("Cross-Validation Accuracy:", round(np.mean(accuracy_scores),3))
    print("Cross-Validation F1 Score:", round(np.mean(f1_scores),3))

    # Fit the best model on the entire training set
    best_estimator.fit(x_train, y_train)

    # Evaluate the model on the test set
    y_pred = best_estimator.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    test_f1_score = f1_score(y_test, y_pred, average='weighted')

    print("Test Set Accuracy:", round(test_accuracy,3))
    print("Test Set F1 Score:", round(test_f1_score,3))

In [22]:
tune_random_forest(heart_df, 'output')

Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 25}
Cross-Validation Accuracy: 0.805
Cross-Validation F1 Score: 0.805
Test Set Accuracy: 0.852
Test Set F1 Score: 0.853


In [23]:
tune_random_forest(stroke_df, 'stroke')

Best Parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 25}
Cross-Validation Accuracy: 0.954
Cross-Validation F1 Score: 0.932
Test Set Accuracy: 0.939
Test Set F1 Score: 0.91


In [24]:
tune_random_forest(water_df, 'Potability')

Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}
Cross-Validation Accuracy: 0.669
Cross-Validation F1 Score: 0.63
Test Set Accuracy: 0.689
Test Set F1 Score: 0.661


# **Logistic Regression**

In [25]:
def tune_logistic_regression(dataset, target_column_name):
    # Split the data into features (x) and the target variable (y)
    x = dataset.drop(columns=[target_column_name])
    y = dataset[target_column_name]

    # Split the data into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    #Scaling the data
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)

    # Create a LogisticRegression classifier
    classifier = LogisticRegression()

    # Define a parameter grid for tuning
    param_grid = {
        'penalty': ['l1', 'l2', 'elasticnet'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        'max_iter': [500, 750, 1000]
    }

    # Create a GridSearchCV object with 5-fold cross-validation
    grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy')

    # Fit the GridSearchCV object to the training data
    grid_search.fit(x_train, y_train)

    # Get the best parameters and estimator from the grid search
    best_params = grid_search.best_params_
    best_estimator = grid_search.best_estimator_

    # Assess the model using 5-fold cross-validation and print accuracy and F1 scores
    accuracy_scores = cross_val_score(best_estimator, x_train, y_train, cv=5, scoring='accuracy')
    f1_scores = cross_val_score(best_estimator, x_train, y_train, cv=5, scoring='f1_weighted')

    print("Best Parameters:", best_params)
    print("Cross-Validation Accuracy:", round(np.mean(accuracy_scores),3))
    print("Cross-Validation F1 Score:", round(np.mean(f1_scores),3))

    # Fit the best model on the entire training set
    best_estimator.fit(x_train, y_train)

    # Evaluate the model on the test set
    y_pred = best_estimator.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    test_f1_score = f1_score(y_test, y_pred, average='weighted')

    print("Test Set Accuracy:", round(test_accuracy,3))
    print("Test Set F1 Score:", round(test_f1_score,3))

In [26]:
tune_logistic_regression(heart_df, 'output')

Best Parameters: {'C': 10, 'max_iter': 500, 'penalty': 'l1', 'solver': 'liblinear'}
Cross-Validation Accuracy: 0.822
Cross-Validation F1 Score: 0.821
Test Set Accuracy: 0.885
Test Set F1 Score: 0.885


In [27]:
tune_logistic_regression(stroke_df, 'stroke')

Best Parameters: {'C': 0.1, 'max_iter': 500, 'penalty': 'l2', 'solver': 'newton-cg'}
Cross-Validation Accuracy: 0.955
Cross-Validation F1 Score: 0.933
Test Set Accuracy: 0.939
Test Set F1 Score: 0.91


In [28]:
tune_logistic_regression(water_df, 'Potability')

Best Parameters: {'C': 0.1, 'max_iter': 500, 'penalty': 'l2', 'solver': 'newton-cg'}
Cross-Validation Accuracy: 0.606
Cross-Validation F1 Score: 0.459
Test Set Accuracy: 0.628
Test Set F1 Score: 0.485


# **k-NN**

In [29]:
def tune_knn(dataset, target_column_name, scaling=True, k_values=[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]):
    # Split the data into features (x) and the target variable (y)
    x = dataset.drop(columns=[target_column_name])
    y = dataset[target_column_name]

    # Split the data into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    #Scaling the data
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)

    # Create a KNeighborsClassifier
    classifier = KNeighborsClassifier()

    # Define a parameter grid for tuning
    param_grid = {
        'n_neighbors': k_values,  # Try different k values
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
    }

    # Create a GridSearchCV object with 5-fold cross-validation
    grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy')

    # Fit the GridSearchCV object to the training data
    grid_search.fit(x_train, y_train)

    # Get the best parameters and estimator from the grid search
    best_params = grid_search.best_params_
    best_estimator = grid_search.best_estimator_

    # Assess the model using 5-fold cross-validation and print accuracy and F1 scores
    accuracy_scores = cross_val_score(best_estimator, x_train, y_train, cv=5, scoring='accuracy')
    f1_scores = cross_val_score(best_estimator, x_train, y_train, cv=5, scoring='f1_weighted')

    print("Best Parameters:", best_params)
    print("Cross-Validation Accuracy:", round(np.mean(accuracy_scores),3))
    print("Cross-Validation F1 Score:", round(np.mean(f1_scores),3))

    # Fit the best model on the entire training set
    best_estimator.fit(x_train, y_train)

    # Evaluate the model on the test set
    y_pred = best_estimator.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    test_f1_score = f1_score(y_test, y_pred, average='weighted')

    print("Test Set Accuracy:", round(test_accuracy,3))
    print("Test Set F1 Score:", round(test_f1_score,3))

In [30]:
tune_knn(heart_df, 'output')

Best Parameters: {'algorithm': 'auto', 'n_neighbors': 8, 'p': 1, 'weights': 'distance'}
Cross-Validation Accuracy: 0.839
Cross-Validation F1 Score: 0.836
Test Set Accuracy: 0.885
Test Set F1 Score: 0.885


In [31]:
tune_knn(stroke_df, 'stroke')

Best Parameters: {'algorithm': 'auto', 'n_neighbors': 9, 'p': 2, 'weights': 'uniform'}
Cross-Validation Accuracy: 0.955
Cross-Validation F1 Score: 0.933
Test Set Accuracy: 0.94
Test Set F1 Score: 0.914


In [32]:
tune_knn(water_df, 'Potability')

Best Parameters: {'algorithm': 'auto', 'n_neighbors': 15, 'p': 1, 'weights': 'distance'}
Cross-Validation Accuracy: 0.655
Cross-Validation F1 Score: 0.621
Test Set Accuracy: 0.646
Test Set F1 Score: 0.615


# **Kaggle**

stevegeorgeparakal

b0fa57c4f837445372ff05a264d63d3e

# **FCA Preprocessing**

In [33]:
# Creating a copy of the dataset for FCA binarization
heart_df_fca = heart_df.copy()

In [34]:
# Binarizing age based on position around the mean
heart_df_fca['age'] = (heart_df_fca['age'] >= heart_df['age'].mean()).astype(int)

# Binarizing chest pain type (cp) whether or not they belong to types (0) (heart related) or (1,2,3) (non heart related)
heart_df_fca['cp'] = heart_df_fca['cp'].apply(lambda x: 1 if x in [0] else 0)

# Binarizing resting blood pressure (trtbps) based on its position around normal blood pressure of humans being 120 mm/Hg
heart_df_fca['trtbps'] = (heart_df_fca['trtbps'] > 120).astype(int)

# Binarizing cholestrol (chol) based on its position around normal cholestrol level which is under 200 mg/dL
heart_df_fca['chol'] = (heart_df_fca['chol'] >= 200).astype(int)

# Binarizing resting electrocardiographic results (restecg) based on whether it is normal (1) or not (0,2)
heart_df_fca['restecg'] = heart_df_fca['restecg'].apply(lambda x: 1 if x in [0, 2] else 0)

# Binarizing maximum heart rate achieved (thalachh) based on theoretical max heart rate calculated by (220-age)
heart_df_fca['thalachh'] = (heart_df_fca['thalachh'] > (220-heart_df['age'])).astype(int)

# Binarizing oldpeak based on position around the mean
heart_df_fca['oldpeak'] = (heart_df_fca['oldpeak'] >= heart_df['oldpeak'].mean()).astype(int)

# Binarizing slope based on flat/positive (1,2) slope and negative slope (0)
heart_df_fca['slp'] = heart_df_fca['slp'].apply(lambda x: 0 if x in [1, 2] else 1)

# Binarizing number of major vessels based on whether there are any or not
heart_df_fca['caa'] = heart_df_fca['caa'].apply(lambda x: 0 if x in [1, 2, 3] else 1)

# Binarizing thalassemia based on whether there is normal bloodflow (2) or not (0,1,3)
heart_df_fca['thall'] = heart_df_fca['thall'].apply(lambda x: 1 if x in [0, 1, 3] else 0)

In [35]:
heart_df_fca.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,1,1,0,1,1,1,1,0,0,1,1,1,1,1
1,0,1,0,1,1,0,0,1,0,1,1,1,0,1
2,0,0,0,1,1,0,1,0,0,1,0,1,0,1
3,1,1,0,0,1,0,0,1,0,0,0,1,0,1
4,1,0,1,0,1,0,0,0,1,0,0,1,0,1


In [36]:
# Creating a copy of the dataset for FCA binarization
stroke_df_fca = stroke_df.copy()

In [37]:
# Binarizing age based on position around the mean
stroke_df_fca['age'] = (stroke_df_fca['age'] >= stroke_df['age'].mean()).astype(int)

# Binarizing average glucose level based on position around the mean
stroke_df_fca['avg_glucose_level'] = (stroke_df_fca['avg_glucose_level'] > stroke_df['avg_glucose_level'].mean()).astype(int)

# Binarizing bmi around the max normal limit 24.9
stroke_df_fca['bmi'] = (stroke_df_fca['bmi'] > 24.9).astype(int)

In [38]:
stroke_df_fca.head()

,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,1,0,1,1,1,1,1,1,0,1,0,0,0,1,0,0,0,1,0,0
1,1,0,0,1,0,1,1,1,1,0,0,0,0,0,1,0,0,0,1,0
2,1,0,1,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,1,0
3,1,0,0,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,1
4,1,1,0,1,0,1,0,1,1,0,0,0,0,0,1,0,0,0,1,0


In [39]:
# Creating a copy of the dataset for FCA binarization
water_df_fca = water_df.copy()

In [40]:
# Binarizing pH based on pH range of drinking water between 6.5 and 8.5
water_df_fca['ph'] = (water_df['ph'] > 6.5) & (water_df['ph'] < 8.5)
water_df_fca['ph'] = water_df_fca['ph'].astype(int)

# Binarizing Hardness based on position around the mean
water_df_fca['Hardness'] = (water_df_fca['Hardness'] < water_df['Hardness'].mean()).astype(int)

# Binarizing Solids based on position around the mean
water_df_fca['Solids'] = (water_df_fca['Solids'] < water_df['Solids'].mean()).astype(int)

# Binarizing Chloramines based on position around the mean
water_df_fca['Chloramines'] = (water_df_fca['Chloramines'] < water_df['Chloramines'].mean()).astype(int)

# Binarizing Sulfate based on position around the mean
water_df_fca['Sulfate'] = (water_df_fca['Sulfate'] < water_df['Sulfate'].mean()).astype(int)

# Binarizing Conductivity based on position around the mean
water_df_fca['Conductivity'] = (water_df_fca['Conductivity'] < water_df['Conductivity'].mean()).astype(int)

# Binarizing Organic Carbon amount based on position around the mean
water_df_fca['Organic_carbon'] = (water_df_fca['Organic_carbon'] < water_df['Organic_carbon'].mean()).astype(int)

# Binarizing Trihalomethanes based on position around the mean
water_df_fca['Trihalomethanes'] = (water_df_fca['Trihalomethanes'] < water_df['Trihalomethanes'].mean()).astype(int)

# Binarizing Turbidity based on position around the mean
water_df_fca['Turbidity'] = (water_df_fca['Turbidity'] < water_df['Turbidity'].mean()).astype(int)

In [41]:
water_df_fca.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,1,0,1,0,0,0,1,0,1,0
1,0,1,1,1,0,0,0,1,0,0
2,1,0,1,0,0,1,0,0,1,0
3,1,0,0,0,0,1,0,0,0,0
4,0,1,1,1,1,1,1,1,0,0


# **FCA Lazy Classification**

In [42]:
def tune_LazyBinaryClassifier(dataset, target_column_name):
  # Split the data into features (x) and the target variable (y)
  x = dataset.drop(columns=[target_column_name])
  y = dataset[target_column_name]

  # Split the data into training and testing sets
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  # Creating a Binarized Classifier
  bin_cls = fcalc.classifier.BinarizedBinaryClassifier(x_train, y_train, method='standard')

  # Predict on train set
  bin_cls.predict(x_train.values)

  # Evaluate the model on the train set
  train_accuracy = accuracy_score(y_train, bin_cls.predictions)
  train_f1_score = f1_score(y_train, bin_cls.predictions, average='weighted')

  print("Train Set Accuracy:", round(train_accuracy,3))
  print("Train Set F1 Score:", round(train_f1_score,3))

  # Predict on test set
  bin_cls.predict(x_test.values)

  # Evaluate the model on the test set
  test_accuracy = accuracy_score(y_test, bin_cls.predictions)
  test_f1_score = f1_score(y_test, bin_cls.predictions, average='weighted')

  print("Test Set Accuracy:", round(test_accuracy,3))
  print("Test Set F1 Score:", round(test_f1_score,3))

In [43]:
tune_LazyBinaryClassifier(heart_df_fca, 'output')

Train Set Accuracy: 0.587
Train Set F1 Score: 0.545
Test Set Accuracy: 0.557
Test Set F1 Score: 0.466


In [44]:
tune_LazyBinaryClassifier(stroke_df_fca, 'stroke')

Train Set Accuracy: 0.15
Train Set F1 Score: 0.193
Test Set Accuracy: 0.165
Test Set F1 Score: 0.196


In [45]:
tune_LazyBinaryClassifier(water_df_fca, 'Potability')

Train Set Accuracy: 0.416
Train Set F1 Score: 0.365
Test Set Accuracy: 0.431
Test Set F1 Score: 0.387
